In [ ]:
%matplotlib widget
# ==== 1) load packages ====
import numpy as np
import pandas as pd
import math
import glob
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import StandardScaler
import torch
import os
os.chdir(
    # "/home/yliu420/Project/Belief-Elicitation-on-LLM-postrelease/analysis"
    "/Users/jingyi/Research/FYP/LLM/conjoint_openai/Belif-Elicitation-on-LLM-postrelease/analysis"
)
from analysis_pkg.preprocess import *
from analysis_pkg.models import *
from PCA_TSNE_scoremodel.visualization import *
from llm_belief.analysis import visualize_probability_distribution
import analysis_pkg.logit_boost as lgt
torch.manual_seed(0)

In [2]:
# ==== 2) load data ====
processes = 10
## load profiles and LLM expectations to profiles and df
fl = [f"../output/{i*1000}_{(i+1)*1000}.csv" for i in range(processes)]
# combine all csv files
df = pd.concat([pd.read_csv(f) for f in fl])
profiles = pd.read_csv("../data/profiles_shuffled.csv")
# ==== 3) preprocess data ====
## split to Xi and Xj by odd and even rows
X_full = preprocess(profiles[:2000*processes])
X_full = X_full.detach().numpy()
P, d = 2, X_full.shape[1]
X = X_full.reshape(-1, P, d)
y = df["profile_id"].apply(lambda x: x % 2) # even -> 0, odd -> 1

/Users/jingyi/Research/FYP/LLM/conjoint_openai/Belif-Elicitation-on-LLM-postrelease/analysis/analysis_pkg/preprocess.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_cols] = X[cat_cols].apply(lambda x: x.astype('category').cat.codes)
/Users/jingyi/Research/FYP/LLM/conjoint_openai/Belif-Elicitation-on-LLM-postrelease/analysis/analysis_pkg/preprocess.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[num_cols] = scaler.fit_transform(X[num_cols])


In [3]:
feature_names = profiles.columns.tolist()
model = lgt.LogitModel()
model.fit(X, y)

Optimized parameters:
[ 0.02136268 -0.27263661  0.18343131 -0.32707681 -0.37133559 -0.17815205
 -2.73723379  0.1834767  -0.15341561  0.05363643]
Maximized LL: -4345.491628546006


In [15]:
# ==== 6) test on real profiles ====
df_real = pd.read_csv("../data/real_profiles.csv")
X_test = df_real[profiles.columns]
X_test = preprocess_test(X_test)
X_test = X_test.detach().numpy()
test_idx = df_real["real model"].tolist()
scores, probs = model.predict_proba(np.vstack([X_full,X_test]))

/Users/jingyi/Research/FYP/LLM/conjoint_openai/Belif-Elicitation-on-LLM-postrelease/analysis/analysis_pkg/preprocess.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_cols] = X[cat_cols].apply(lambda x: x.astype('category').cat.codes)
/Users/jingyi/Research/FYP/LLM/conjoint_openai/Belif-Elicitation-on-LLM-postrelease/analysis/analysis_pkg/preprocess.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[num_cols] = scaler.transform(X[num_cols])


In [17]:
# print the most likely profile
print(profiles.iloc[scores.argmax()])
print('score: ',scores.max())
print("\n====real profiles====\n")
ranks = scores.argsort()[::-1]
for p, i in zip(range(len(ranks)-8, len(ranks)), test_idx):
    rank = np.where(ranks==p)[0][0]+1
    print(f"{i}'s score: {scores[p]:.2f}, rank: {rank}/{len(ranks)}, percentile: {100*(len(ranks)-rank)/len(ranks):.2f}%")

battery life (in hours of video playback)        18.0
screen size (in inches)                           6.1
thickness (in mm)                                 8.8
front camera resolution (in MP)                  11.0
rear camera main lens resolution (in MP)         36.0
rear camera longest focal length (in x)           3.0
ultrawide camera                             equipped
Geekbench multicore score                      8600.0
RAM                                                 4
price                                           959.0
Name: 15941, dtype: object
score:  1.9226798840359896

====real profiles====

iPhone 16's score: 1.22, rank: 326/20008, percentile: 98.37%
iPhone 16 Plus's score: 0.73, rank: 1482/20008, percentile: 92.59%
iPhone 16 Pro's score: 1.09, rank: 501/20008, percentile: 97.50%
iPhone 16 Pro Max's score: 0.59, rank: 2078/20008, percentile: 89.61%
iPhone 17's score: 0.89, rank: 940/20008, percentile: 95.30%
iPhone 17 Air's score: -2.45, rank: 13379/20008, percentil